## Simple overhead benchmark

CPython 3.6.3 and an i7 7700K were used for the timings

In [1]:
from antidote import antidote
import attr

container = antidote.container

In [2]:
@antidote.register
class Service1:
    pass


@antidote.register
class Service2:
    def __init__(self, service1: Service1):
        self.service1 = service1
       
 
@antidote.register
class Service3:
    def __init__(self, service1: Service1, service2: Service2):
        self.service1 = service1
        self.service2 = service2

  
@antidote.register
class Service4:
    def __init__(self, service1: Service1, service2: Service2, service3: Service3):
        self.service1 = service1
        self.service2 = service2
        self.service3 = service3

### Function call

In [3]:
def f(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    return s1, s2, s3, s4

%timeit f(container[Service1], container[Service2], container[Service3], container[Service4])

673 ns ± 2.11 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
f_injected = antidote.inject(f)
%timeit f_injected()

1.77 µs ± 21.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
%timeit f_injected(container[Service1], container[Service2], container[Service3], container[Service4])

1.48 µs ± 23.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [6]:
f_bound = antidote.inject(f, bind=True)

%timeit f_bound()

208 ns ± 0.56 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Object instantiation

In [7]:
class Obj:
    s1: Service1
    s2: Service2
    s3: Service3
    s4: Service4
        
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit Obj(container[Service1], container[Service2], container[Service3], container[Service4])

940 ns ± 8.97 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
class ObjInjected:
    s1: Service1
    s2: Service2
    s3: Service3
    s4: Service4
        
    @antidote.inject
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit ObjInjected()

2.05 µs ± 48.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
@attr.s
class ObjAttrs:
    s1: Service1 = antidote.attrib()
    s2: Service2 = antidote.attrib()
    s3: Service3 = antidote.attrib()
    s4: Service4 = antidote.attrib()
        
%timeit ObjAttrs()

1.46 µs ± 7.75 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
